In [5]:
# Импорт необходимых библиотек
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
import os
import numpy as np

# Настройка отображения
pd.set_option('display.max_columns', 50)
plt.style.use('seaborn')
sns.set_palette("Set2")

# 1. Загрузка данных
print("="*50)
print("Загрузка данных...")
try:
    # Проверка существования файлов (с учетом ваших имен файлов)
    required_files = {
        'DEMO': 'DEMO_Jxpt',
        'LABS': 'BIOPRO_Jxpt',
        'MORT': 'NHS_2017_MORT_2019_PUBLIC.dat'
    }
    
    for file_type, file_name in required_files.items():
        if not os.path.exists(file_name):
            raise FileNotFoundError(f"Файл {file_name} не найден в рабочей директории")
    
    # Загрузка данных с учетом ваших имен файлов
    print("\nЗагрузка демографических данных...")
    demo = pd.read_sas(required_files['DEMO'])
    
    print("Загрузка лабораторных данных...")
    labs = pd.read_sas(required_files['LABS'])
    
    print("Загрузка данных о смертности...")
    mort = pd.read_fwf(
        required_files['MORT'],
        colspecs=[(0, 7), (8, 9), (10, 11), (12, 18), (19, 25)],
        names=["SEQN", "ELIGSTAT", "MORTSTAT", "PERMTH_INT", "PERMTH_EXM"],
        dtype={"SEQN": int, "ELIGSTAT": int, "MORTSTAT": int}
    )
    
    print("\n" + "="*50)
    print("Данные успешно загружены:")
    print(f"- Демография: {demo.shape[0]} записей, {demo.shape[1]} переменных")
    print(f"- Лаборатории: {labs.shape[0]} записей, {labs.shape[1]} переменных")
    print(f"- Смертность: {mort.shape[0]} записей")
    print("Пример SEQN из каждого файла:")
    print(f"Демо: {demo['SEQN'].iloc[0]}, Лабы: {labs['SEQN'].iloc[0]}, Смертность: {mort['SEQN'].iloc[0]}")
    
except Exception as e:
    print("\n" + "="*50)
    print(f"Ошибка загрузки данных: {str(e)}")
    print(f"Текущая рабочая директория: {os.getcwd()}")
    print("Доступные файлы:", [f for f in os.listdir() if f.endswith(('.xpt','.dat', '.XPT'))])
    exit()

# 2. Объединение и очистка данных
print("\n" + "="*50)
print("Объединение и очистка данных...")
try:
    # Пошаговое объединение с проверкой
    print("\nШаг 1: Объединение демографии и лабораторий...")
    data = pd.merge(demo, labs, on="SEQN", how="inner", validate="one_to_one")
    print(f"После первого слияния: {data.shape[0]} записей")
    
    print("Шаг 2: Добавление данных о смертности...")
    data = pd.merge(data, mort, on="SEQN", how="left", validate="one_to_one")
    print(f"После второго слияния: {data.shape[0]} записей")
    
    # Фильтрация и преобразование
    print("\nФильтрация данных...")
    initial_count = data.shape[0]
    data = data[data["ELIGSTAT"] == 1].copy()
    data["MORTSTAT"] = data["MORTSTAT"].fillna(0).astype(int)
    print(f"Отфильтровано записей: {initial_count} → {data.shape[0]}")
    
    # Поиск бикарбоната (с учетом возможных названий)
    bicarb_names = {
        'LBXBEC': 'Бикарбонат сыворотки (стандартное название)',
        'LBXHCO3': 'Бикарбонат (альтернативное название)',
        'LBDHCO3': 'Бикарбонат (еще один вариант)',
        'BIOCARB': 'Возможное название в ваших данных'
    }
    
    found = False
    for col in bicarb_names:
        if col in data.columns:
            data.rename(columns={col: "BICARBONATE"}, inplace=True)
            print(f"\nНайден столбец бикарбоната: {col} → переименован в BICARBONATE")
            found = True
            break
    
    if not found:
        # Если бикарбонат не найден, попробуем найти другие показатели кислотности
        acid_cols = [col for col in data.columns if 'ph' in col.lower() or 'acid' in col.lower()]
        if acid_cols:
            print(f"\nСтолбец бикарбоната не найден, но найдены потенциальные альтернативы: {acid_cols}")
            data.rename(columns={acid_cols[0]: "ACID_MEASURE"}, inplace=True)
            print(f"Используем столбец {acid_cols[0]} как показатель кислотности")
        else:
            raise KeyError(f"Столбец с бикарбонатом не найден. Доступные столбцы: {list(data.columns)}")
    
    # Очистка от пропущенных значений
    initial_count = data.shape[0]
    target_col = "BICARBONATE" if "BICARBONATE" in data.columns else "ACID_MEASURE"
    data.dropna(subset=[target_col, "MORTSTAT"], inplace=True)
    print(f"\nУдалено записей с пропущенными значениями: {initial_count} → {data.shape[0]}")
    
    # Добавим категоризацию по возрасту (если есть столбец возраста)
    if 'RIDAGEYR' in data.columns:
        data['AGE_GROUP'] = pd.cut(data['RIDAGEYR'],
                                  bins=[0, 30, 45, 60, 75, 100],
                                  labels=['<30', '30-44', '45-59', '60-74', '75+'])
        age_info = " (с категоризацией по возрасту)"
    else:
        age_info = " (столбец возраста не найден)"
    
    print("\nПервые 3 записи:")
    print(data.iloc[:3, :10])  # Показать первые 3 строки и 10 столбцов
    
    print("\nРаспределение по статусу смертности:")
    print(data["MORTSTAT"].value_counts(normalize=True).mul(100).round(1).astype(str) + '%')
    
except Exception as e:
    print("\n" + "="*50)
    print(f"Ошибка обработки данных: {str(e)}")
    if 'data' in locals():
        print("\nДоступные столбцы:", list(data.columns))
        print("\nПример данных:")
        print(data.iloc[:3, :10])  # Первые 3 строки и 10 столбцов
    exit()

# 3. Анализ данных
print("\n" + "="*50)
print("Анализ данных...")

target_col = "BICARBONATE" if "BICARBONATE" in data.columns else "ACID_MEASURE"
print(f"\nАнализируем показатель: {target_col}")

# Описательная статистика с группировкой
print("\nОписательная статистика:")
desc_stats = data.groupby("MORTSTAT")[target_col].describe().round(2)
print(desc_stats)

# 4. Визуализация
print("\n" + "="*50)
print("Создание визуализаций...")
plt.figure(figsize=(14, 6))

# График 1: Распределение показателя
plt.subplot(1, 2, 1)
sns.boxplot(x="MORTSTAT", y=target_col, data=data)
plt.title(f"Распределение {target_col}\nпо статусу смертности", fontsize=12)
plt.xlabel("Статус смертности (0 = жив, 1 = умер)", fontsize=10)
plt.ylabel(target_col, fontsize=10)

# График 2: Гистограмма распределения
plt.subplot(1, 2, 2)
sns.histplot(data=data, x=target_col, hue="MORTSTAT", element="step", stat="density", common_norm=False)
plt.title(f"Распределение {target_col}", fontsize=12)
plt.xlabel(target_col, fontsize=10)
plt.ylabel("Плотность", fontsize=10)
plt.legend(title="Статус", labels=["Живы", "Умерли"])

plt.tight_layout()
plt.savefig("analysis_result.png", dpi=300, bbox_inches='tight')
print("\nГрафики сохранены в analysis_result.png")

# 5. Статистический анализ
print("\n" + "="*50)
print("Проведение статистического анализа...")

# Проверка нормальности распределения
print("\nПроверка нормальности распределения:")
norm_test = pd.DataFrame(columns=['Группа', 'Статистика', 'p-value'])

for status in [0, 1]:
    sample = data[data["MORTSTAT"] == status][target_col].sample(min(5000, len(data[data["MORTSTAT"] == status])))
    stat, p = stats.shapiro(sample)
    norm_test.loc[len(norm_test)] = [
        "Живые" if status == 0 else "Умершие",
        round(stat, 4),
        round(p, 6)
    ]

print(norm_test)

# Выбор теста
if norm_test.loc[0, 'p-value'] > 0.05 and norm_test.loc[1, 'p-value'] > 0.05:
    # t-тест для нормальных распределений
    t_stat, p_value = stats.ttest_ind(
        data[data["MORTSTAT"] == 0][target_col],
        data[data["MORTSTAT"] == 1][target_col],
        equal_var=False  # Тест Уэлча
    )
    test_name = "t-тест (Уэлча)"
else:
    # U-тест Манна-Уитни для ненормальных распределений
    t_stat, p_value = stats.mannwhitneyu(
        data[data["MORTSTAT"] == 0][target_col],
        data[data["MORTSTAT"] == 1][target_col]
    )
    test_name = "U-тест Манна-Уитни"

print(f"\nРезультаты {test_name}:")
print(f"Статистика = {t_stat:.4f}, p-value = {p_value:.6f}")

# Эффект размера
def cohens_d(x, y):
    nx = len(x)
    ny = len(y)
    dof = nx + ny - 2
    return (x.mean() - y.mean()) / np.sqrt(((nx-1)*x.std()**2 + (ny-1)*y.std()**2) / dof)

effect_size = cohens_d(
    data[data["MORTSTAT"] == 0][target_col],
    data[data["MORTSTAT"] == 1][target_col]
)

print(f"\nРазмер эффекта (Коэна d): {abs(effect_size):.3f}")
print("Интерпретация размера эффекта:")
print("- 0.2: маленький эффект")
print("- 0.5: средний эффект")
print("- 0.8: большой эффект")

print("\n" + "="*50)
print("Итоговые выводы:")
if p_value < 0.05:
    print(f"1. Обнаружена статистически значимая разница (p = {p_value:.4f})")
    print(f"2. Размер эффекта: {abs(effect_size):.3f} ({'малый' if abs(effect_size) < 0.5 else 'средний' if abs(effect_size) < 0.8 else 'большой'})")
else:
    print("1. Статистически значимых различий не обнаружено (p ≥ 0.05)")

print("\nАнализ успешно завершен!")
print("="*50)

ModuleNotFoundError: No module named 'scipy'